```
dir=/shared_volume/
#dir=/LUSTRE/MADMEX/tasks/2021/pyei_pipeline

mkdir $dir/pyei_data

wget -O $dir/pyei_data/dropbox_data.zip https://www.dropbox.com/sh/alab97y1fimlzoe/AADcNaR-Y66IfelP0nqnN0Bca?dl=1

sudo apt-get install unzip

unzip $dir/pyei_data/dropbox_data.zip -d $dir/pyei_data/

```

Must use 16gb for docker container [sipecam/madmex-kale:0.5.0_0.1.0](https://github.com/CONABIO/kube_sipecam/blob/master/minikube_sipecam/deployments/MAD_Mex/jupyterlab-mad-mex-0.5.0_0.1.0.yaml#L19)

In [1]:
import math

import os
import ntpath
import numpy as np
import fiona
import rasterio
from sklearn.ensemble import RandomForestRegressor
import gc
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly 
from osgeo.gdalconst import GDT_Float32
from osgeo.gdalconst import GDT_Int16
from osgeo.gdalconst import GDT_UInt16

In [2]:
#direc = "/LUSTRE/MADMEX/tasks/2021/pyei_pipeline"
direc = "/shared_volume"
data_path = os.path.join(direc,"pyei_data", "1_strucdiv")
infys_shapefile = os.path.join(direc,"pyei_data", "1_strucdiv", "1.1_strucdiv_infys.shp")
train_data_dir = os.path.join(direc, "pyei_train_data")
train_file = "1.1_train_table.csv"
pyei_results_dir = os.path.join(direc, "pyei_results")

In [3]:
# Make a list of files with a certain ending.
# Search is recursive and traverses all (sub)folders.
def list_files(directory, endswith=".tif"):
    files_endswith = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(endswith):
                fullpath = root + '/' + file
                files_endswith.append(fullpath)
    return files_endswith

# Obtain point coordinates from a point feature in a shapefile.
def coords(point):
    x = point["geometry"]["coordinates"][0]
    y = point["geometry"]["coordinates"][1]

    return {"coordinates" : (x, y)}

# Obtain the data associated to a variable of a shapefile.
def values(points, variable):
    npoints = len(points)
    nvalues = np.zeros(npoints)
    i=0
    for point in points:
        val = point["properties"][variable]
        nvalues[i] = val
        i += 1
    return(nvalues)


# Given a shapefile read with fiona extract raster values.
def extract(points, raster):

    npoints = len(points)
    ex_values = np.zeros(npoints)
    i=0
    with rasterio.open(raster) as src:
        for point in points:
            for value in src.sample([coords(point)["coordinates"]]):
                ex_values[i] = value
                i += 1
    return ex_values


# From a pair (spatial_sampling_points, raster_variable_list)
# Create a traininig table for model building.
# First column is considered the dependent variable.
def create_trainset(points, file_list, variable):

    trainset = np.empty((len(points), len(file_list) + 1))
    vals = values(points = points, variable = variable)
    trainset[:,0] = values(points = points, variable = variable)

    i=1
    for file in file_list: 
        trainset[:,i] = extract(points = points, raster = file)
        i += 1 

    return(trainset) 

# Get raster dimensions.
def dims(raster):
    with rasterio.open(raster, 'r') as ds:
        raster = ds.read()

    raster_dimensions = raster.shape    
    x = raster_dimensions[1]
    y = raster_dimensions[2]
    return (x,y)

# Given a raster file-list create a numpy array.
# Each raster (variable) becomes a column.
def rasters_to_table(file_list):

    # Read in first raster to extract metadata.
    xy = dims(file_list[0])
 
    x = xy[0]
    y = xy[1]

    # Initialize numpy array to place raster bands.
    ntable = np.zeros((x*y, len(file_list)))
    i = 0
    for raster in file_list:
        with rasterio.open(raster, 'r') as ds:
            ntable[:,i] = ds.read().flatten()
            i += 1

    return(ntable)

In [4]:
output_file_train_data = os.path.join(train_data_dir, train_file)



if not os.path.exists(output_file_train_data):
    
    os.mkdir(train_data_dir)

    geotiffs = list_files(data_path)
    
    shapef_source = os.path.join(data_path, infys_shapefile)
    
    shapef = fiona.open(shapef_source)
    
    train_table = create_trainset(shapef, geotiffs, "AlturTtl_m")
    
    np.savetxt(output_file_train_data, train_table, delimiter=',')

In [5]:
# Load training data array.
data = np.loadtxt(output_file_train_data, delimiter=',')

# Instantiate model with 1000 decision trees.
# n_jobs is subject to available cores (8 cores in parallel in this case).
model = RandomForestRegressor(n_estimators = 100, random_state = 42, oob_score = True, n_jobs = 8)

# Train the model.
model.fit(data[:,1:], data[:,0])

# Out-Of-Bag estimation of correlatino between observed and predicted values.
oob_corr = math.sqrt(model.oob_score_)
print(oob_corr)

0.5123075273950167


In [6]:
# Use the forest's predict method on the full data set (all of MX).
geotiffs = list_files(data_path)

rast = rasterio.open(geotiffs[0])

rmetadata = rast.meta

full_data = rasters_to_table(geotiffs)
predictions = model.predict(full_data).astype("float32")
predictions = predictions.reshape((rmetadata["height"], rmetadata["width"]))

predictions_file = os.path.join(pyei_results_dir, "1.2_avg_tree_height.tif")

if not os.path.exists(pyei_results_dir):
    os.mkdir(pyei_results_dir)
with rasterio.open(predictions_file, 'w', **rmetadata) as dst:
    dst.write(predictions, indexes = 1)

In [7]:
def readtif(imagepath):
    gdal.AllRegister()
    inDataset = gdal.Open(imagepath,GA_ReadOnly)
    cols = inDataset.RasterXSize
    rows = inDataset.RasterYSize
    bands = inDataset.RasterCount
    return(inDataset,rows,cols,bands)

def createtif(driver,rows,cols,bands,outpath,data_type=32):
    if data_type==32:
        outDataset = driver.Create(outpath,cols,rows,bands,GDT_Float32,[ "COMPRESS=LZW" ])
    elif data_type==16:
        outDataset = driver.Create(outpath,cols,rows,bands,GDT_UInt16,[ "COMPRESS=LZW" ])
    return(outDataset)

def writetif(outDataset,data,projection,geotransform,order='r'):
    # order controls if the columns or the rows should be considered the observations
    cols = outDataset.RasterXSize 
    rows = outDataset.RasterYSize
    if geotransform is not None:
        gt = list(geotransform)
        gt[0] = gt[0] + 0*gt[1]
        gt[3] = gt[3] + 0*gt[5]
        outDataset.SetGeoTransform(tuple(gt))
    if projection is not None:
        outDataset.SetProjection(projection)
    
    if data.ndim==1:
        outBand = outDataset.GetRasterBand(1)
        resized = np.reshape(data,(rows,cols))
        outBand.WriteArray(resized,0,0)
        outBand.FlushCache()
    else:
        if order=='r':
            n=np.shape(data)[0]
            for k in range(n):
                outBand = outDataset.GetRasterBand(k+1)
                outBand.WriteArray(np.resize(data[k,:],(rows,cols)),0,0)
                outBand.FlushCache()
        elif order=='c':
            n=np.shape(data)[1]
            for k in range(n):
                outBand = outDataset.GetRasterBand(k+1)
                outBand.WriteArray(np.reshape(data[:,k],(rows,cols)))
                outBand.FlushCache()

    #close the dataset properly
    outDataset = None

def swapValues(flattenedNumpyArray,listOfInLists,listOfSwappingValues):
    '''
    takes each list in tlistOfInLists and swaps it by the
    corresponding value in listOfSwappingValues
    '''
    aux=flattenedNumpyArray
    if len(listOfInLists)!=len(listOfSwappingValues):
        print("Lists must be of the same length.")
    else:
        for i in range(len(listOfInLists)):
            # list to numpy array
            nparray = np.array(listOfInLists[i])
            found_idx = np.in1d(flattenedNumpyArray,nparray)
            aux[found_idx]=listOfSwappingValues[i]
    aux = aux.astype(int)
    return(aux)
def multispectralToBits(multispec_raster_path,class_ids, out_dir):
    # Get data from raster with classifications
    ds = gdal.Open(multispec_raster_path)
    band = ds.GetRasterBand(1)
    class_ar = band.ReadAsArray()
    gt = ds.GetGeoTransform()
    pj = ds.GetProjection()
    ds = band = None  # close
    
    # Make a new bit rasters
    drv = gdal.GetDriverByName("GTiff")
    bit_raster_file = os.path.join(pyei_results_dir, "bit_raster.tif")
    ds = drv.Create(bit_raster_file, class_ar.shape[1], class_ar.shape[0],
                    len(class_ids), gdal.GDT_Byte, ["NBITS=1"])
    ds.SetGeoTransform(gt)
    ds.SetProjection(pj)
    for bidx in range(ds.RasterCount):
        band = ds.GetRasterBand(bidx + 1)
        # create boolean result where 0 == no and 1 == yes
        selection = (class_ar == class_ids[bidx]).astype("u1")
        band.WriteArray(selection)
    ds = band = None  # save, close
    return bit_raster_file

In [8]:
gdal.UseExceptions()

data_path = os.path.join(direc,"pyei_data","2_landcover")
in_raster = "madmex_lcc_landsat_2018_v4.3.1_chp.tif"
out_raster = "madmex_lcc_landsat_2018_v4.3.1_chp_7c.tif"


In [9]:
in_raster_file = os.path.join(data_path, in_raster)
dataset,rows,cols,bands = readtif(in_raster_file)

# Image metadata.
projection = dataset.GetProjection()
transform = dataset.GetGeoTransform()
driver = dataset.GetDriver()

# make numpy array and flatten
band = dataset.GetRasterBand(1)
band = band.ReadAsArray(0, 0, cols, rows).astype(np.int16)
band = np.ravel(band)

# Remove missing values from class remapping.
gooddata_idx = band != 0
gooddata = band[gooddata_idx]

# Raster with remapped classes.
aggregated = swapValues(gooddata,[\
                                  [1,2,3,6],\
                                  [7,8, 9,10,11,12, 21, 22, 25, 26],\
                                  #[4,5, 13, 14, 15, 16, 17, 18, 19, 20],\
                                  [27, 28],\
                                  [23, 24, 30],\
                                  [29],\
                                  [31]
                                 ],\
                        [1,2,4,5,6,7])

# [1,2,3,8] 						  -to- 1 bosque
# [9,10,11,12,13,14,15,16]    		  -to- 2 selvas
# [4,5,6,7,17,18,19,21,22,23, 25, 26] -to- 3 matorrales
# [27, 28] 							  -to- 4 pastizal y agricultura 
# [30, 20, 24]  					  -to- 5 suelo desnudo
# [31]      						  -to- 6 asentamiento humano
# [29]      						  -to- 7 agua
# [98, 99]  						  -to- 8 nieve y hielo

band[gooddata_idx] = aggregated

# Set up output and write.
tif_out_file = os.path.join(pyei_results_dir, out_raster)
if not os.path.exists(tif_out_file):
    outData = createtif(driver, rows, cols, 1, tif_out_file,16)
    writetif(outData,band, projection, transform)
    outData = None


In [10]:
in_raster = "madmex_lcc_landsat_2018_v4.3.1_chp_7c.tif"
out_raster = "madmex_lcc_landsat_2018_v4.3.1_chp_7cprop.tif"

In [11]:
# Open raster.
class_ids=[1,2,4,5,6,7]

data_to_bits_file = os.path.join(pyei_results_dir, in_raster)
res_multispectraltobits = multispectralToBits(data_to_bits_file, class_ids, pyei_results_dir)

src_ds = gdal.Open(res_multispectraltobits)

# Open a template or copy array, for dimensions and NODATA mask.
cpy_ds_file = os.path.join(pyei_results_dir, "1.2_avg_tree_height.tif")
cpy_ds = gdal.Open(cpy_ds_file)
band = cpy_ds.GetRasterBand(1)
cpy_mask = (band.ReadAsArray() == band.GetNoDataValue())

# Result raster, with same resolution and position as the copy raster.
drv = gdal.GetDriverByName("GTiff")

#out_raster = "madmex_lcc_landsat_2018_v4.3.1_chp_7c.tif"

dst_ds_file = os.path.join(pyei_results_dir, out_raster)

if not os.path.exists(dst_ds_file):

    dst_ds = drv.Create(dst_ds_file, cpy_ds.RasterXSize, cpy_ds.RasterYSize,
                        len(class_ids), gdal.GDT_Float32, ["INTERLEAVE=BAND"])
    dst_ds.SetGeoTransform(cpy_ds.GetGeoTransform())
    dst_ds.SetProjection(cpy_ds.GetProjection())
    
    # Do the same as gdalwarp -r average; this might take a while to finish.
    gdal.ReprojectImage(src_ds, dst_ds, None, None, gdal.GRA_Average)
    
    # Convert all fractions to percent, and apply the same NODATA mask from the copy raster.
    NODATA = 0
    for bidx in range(dst_ds.RasterCount):
        band = dst_ds.GetRasterBand(bidx + 1)
        ar = band.ReadAsArray() * 100.0
        ar[cpy_mask] = NODATA
        band.WriteArray(ar)
        band.SetNoDataValue(NODATA)
    
    # Save and close all rasters
    src_ds = cpy_ds = dst_ds = band = None